In [1]:
pwd()

'/data1/home/mrim/gonzagab/Kodicare'

In [11]:
import numpy as np
import pandas as pd
import pyterrier as pt
if not pt.started():
    pt.init(mem=8000)
pt.set_tqdm('tqdm')
import xml.etree.ElementTree as ET
import json
import random
import math
import ir_datasets
import os

# Robust Collection (TREC45 (Ad hoc retrieval))

In [5]:
robust_path = "/data1/home/mrim/galuscap/data/robust/disk45"
robust_index_path = "/data1/home/mrim/galuscap/data/indexes/robust"

In [ ]:
if os.path.exists(os.path.join(robust_index_path, "data.properties")):
    robust_indexref = pt.IndexRef.of(os.path.join(robust_index_path, "data.properties"))
else:    
    files = pt.io.find_files(robust_path)
    # no-one indexes the congressional record in directory /CR/
    # indeed, recent copies from NIST dont contain it
    # we also remove some of the other unneeded files
    bad = ['/CR/', '/AUX/', 'READCHG', 'READMEFB', 'READFRCG', 'READMEFR', 'READMEFT', 'READMELA']
    for b in bad:
        files = list(filter(lambda f: b not in f, files))
    robust_indexer = pt.TRECCollectionIndexer(robust_index_path, verbose=True)
    robust_indexref = indexer.index(files)
    # processing the files took 7 minutes; the total indexing process took 7m40

robust_index = pt.IndexFactory.of(robust_indexref)
print(robust_index.getCollectionStatistics().toString())

In [16]:
BM25 = pt.BatchRetrieve(index, wmodel="BM25")
DPH  = pt.BatchRetrieve(index, wmodel="DPH")
PL2  = pt.BatchRetrieve(index, wmodel="PL2")
DLM  = pt.BatchRetrieve(index, wmodel="DirichletLM")

In [17]:
pt.Experiment(
    [BM25, DPH, PL2, DLM],
    pt.get_dataset("trec-robust-2004").get_topics(),
    pt.get_dataset("trec-robust-2004").get_qrels(),
    eval_metrics=["map", "P_10", "P_20", "ndcg_cut_20"],
    names=["BM25", "DPH", "PL2", "Dirichlet QL"]
)





04.testset.gz:   0%|                                            | 0.00/33.5k [00:00<?, ?iB/s]



04.testset.gz: 100%|████████████████████████████████████| 33.5k/33.5k [00:00<00:00, 260kiB/s]
  0%|                                                            | 0/2306 [31:21<?, ?files/s]


qrels.robust2004.txt: 100%|████████████████████████████| 6.24M/6.24M [00:01<00:00, 5.62MiB/s]
  0%|                                                            | 0/2307 [35:53<?, ?files/s]


,name,map,P_10,P_20,ndcg_cut_20
0,BM25,0.241763,0.426104,0.349398,0.408061
1,DPH,0.251307,0.447390,0.361446,0.422524
2,PL2,0.229383,0.420884,0.343775,0.402179
3,Dirichlet QL,0.236826,0.407631,0.337952,0.396870


# CAR Collection 

List of collections can be found here: https://pyterrier.readthedocs.io/en/latest/datasets.html

Description of CAR 1.5 (https://trec-car.cs.unh.edu/datareleases/v1.5-release.html) -- we might need v1.7 which is not supported by Terrier? 

It actually points to this: https://ir-datasets.com/car.html

In [ ]:
car_dataset = "car/v1.5"
car_index_path = "/data1/home/mrim/galuscap/data/indexes/car"

dataset = ir_datasets.load(car_dataset)
for doc in dataset.docs_iter():
    doc # namedtuple<doc_id, text>
    


In [6]:
dataset = pt.datasets.get_dataset('car/v1.5')
indexer = pt.index.IterDictIndexer('./car-index')
indexref = indexer.index(dataset.get_corpus_iter(), fields=('text', 'title', 'headings'))
index = pt.IndexFactory.of(indexref)

DPH_br = pt.BatchRetrieve(index, wmodel="DPH") % 100
BM25_br = pt.BatchRetrieve(index, wmodel="BM25") % 100
# this runs an experiment to obtain results on the TREC COVID queries and qrels
pt.Experiment(
    [DPH_br, BM25_br],
    dataset.get_topics('title'),
    dataset.get_qrels(),
    eval_metrics=["P.5", "P.10", "ndcg_cut.10", "map"])

#indexer = pt.TRECCollectionIndexer(car_index_path)
# this downloads the file msmarco-docs.trec.gz
#indexref = indexer.index(car_dataset.get_corpus())
#index = pt.IndexFactory.of(indexref)

#DPH_br = pt.BatchRetrieve(index, wmodel="DPH") % 100
#BM25_br = pt.BatchRetrieve(index, wmodel="BM25") % 100
# this runs an experiment to obtain results on the TREC 2019 Deep Learning track queries and qrels
#pt.Experiment(
#    [DPH_br, BM25_br],
#    dataset.get_topics("test"),
#    dataset.get_qrels("test"),
#    eval_metrics=["recip_rank", "ndcg_cut_10", "map"])

KeyError: 'car/v1.5'

# trec-wt10g

In [7]:
files_wt10g = pt.io.find_files("/home/mrim/galuscap/data/wt10g")

# build the index
indexer_wt10g = pt.TRECCollectionIndexer("/home/mrim/galuscap/data/indexes/wt10g_index", verbose=True, blocks=False)
indexref_wt10g = indexer_wt10g.index(files_wt10g)

# load the index, print the statistics
index_wt10g = pt.IndexFactory.of(indexref_wt10g)
print(index_wt10g.getCollectionStatistics().toString())


100%|████████████████████████████████████████████████▉| 5158/5159 [27:00<00:00,  3.65files/s]

14:39:19.896 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/wt10g/qrels.adhoc.451-550.txt. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed


  0%|                                                            | 0/5159 [28:20<?, ?files/s]

14:39:19.916 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/wt10g/topics.adhoc.451-550.txt. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed




5160files [27:01,  3.18files/s]                                                              


Number of documents: 1692096
Number of terms: 3135780
Number of postings: 279636926
Number of fields: 0
Number of tokens: 673375016
Field names: []
Positions:   false



In [14]:
# These are the queries from TREC10 ad-hoc collection -- another options are trec 9 and trec9, trec10-adhoc, trec10-hp 

dataset_wt10g = pt.get_dataset("trec-wt10g")
bm25 = pt.BatchRetrieve(index_wt10g, wmodel="BM25")
dph = pt.BatchRetrieve(index_wt10g, wmodel="DPH")
pt.Experiment(
    [bm25, dph],
    dataset_wt10g.get_topics("trec10-adhoc"),
    dataset_wt10g.get_qrels("trec10-adhoc"),
    eval_metrics=["map"]
)

qrels.trec10.main_web.gz: 100%|███████████████████████████| 277k/277k [00:00<00:00, 550kiB/s]


,name,map
0,BR(BM25),0.191039
1,BR(DPH),0.214665


# TREC12 (Ad hoc retrieval)

More on this: https://github.com/diazf/trec-data (Tipster collection)

In [19]:
files_disk12 = pt.io.find_files("/home/mrim/galuscap/data/disk12")

# build the index
indexer_disk12 = pt.TRECCollectionIndexer("/home/mrim/galuscap/data/indexes/disk12", verbose=True, blocks=False)
indexref_disk12 = indexer_disk12.index(files_disk12)

# load the index, print the statistics
index_disk12 = pt.IndexFactory.of(indexref_disk12)
print(index_disk12.getCollectionStatistics().toString())

# There were errors here !!! Is this dataset complet?

 19%|█████████▋                                        | 462/2394 [02:56<25:50,  1.25files/s]

15:55:51.162 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (DOE1-96-1081) - further warnings are suppressed


 23%|███████████▍                                      | 547/2394 [03:53<18:00,  1.71files/s]

15:56:47.910 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/disk12/disk1/dtd/ap.dtd. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
15:56:47.943 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/disk12/disk1/dtd/doe.dtd. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
15:56:47.977 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/disk12/disk1/dtd/fr.dtd. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
15:56:47.999 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/disk12/disk1/dtd/wsj.dtd. Perhaps trec.collection.class is wrongly set, TrecDocTa

 31%|███████████████▋                                  | 749/2394 [04:54<05:33,  4.93files/s]

...

 92%|█████████████████████████████████████████████▎   | 2213/2394 [07:43<00:07, 24.81files/s]

16:00:37.562 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/disk12/disk2/ziff/zf_251.z. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
16:00:37.618 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/disk12/disk2/ziff/zf_252.z. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
16:00:37.658 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/disk12/disk2/ziff/zf_253.z. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
16:00:37.697 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/disk12/disk2/ziff/zf_254.z. Perhaps trec.collection.class is wrongly set,

2395files [07:53,  5.06files/s]                                                              

16:00:47.272 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


16:01:02.807 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key DOE1-96-0001 is not unique: 336728,310766
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:369)
Number of documents: 436517
Number of terms: 415734
Number of postings: 53075626
Number of fields: 0
Number of tokens: 104889119
Field names: []
Positions:   false



# Clueweb 09 and Clueweb12 (WebTrack)

# Gov2

In [ ]:
files_gov2 = pt.io.find_files("/home/mrim/galuscap/data/gov2/gov2-corpus")

# build the index
indexer_gov2 = pt.TRECCollectionIndexer("/home/mrim/galuscap/data/indexes/gov2", verbose=True, blocks=True)
indexref_gov2 = indexer_gov2.index(files_gov2)

# load the index, print the statistics
index_gov2 = pt.IndexFactory.of(indexref_gov2)
print(index_gov2.getCollectionStatistics().toString())


  0%|                                                           | 0/27205 [00:00<?, ?files/s]

14:35:35.621 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (GX000-00-0270264) - further warnings are suppressed



  1%|▌                                             | 301/27205 [08:42<16:23:50,  2.19s/files]

14:44:20.326 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/gov2/gov2-corpus/GX003/.nfs0000002e1dbafe0d000000ac. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed



 16%|██████▋                                    | 4220/27205 [1:59:25<11:05:34,  1.74s/files]

In [ ]:
DPH_br = pt.BatchRetrieve(index, wmodel="DPH") % 100
BM25_br = pt.BatchRetrieve(index, wmodel="BM25") % 100
# this runs an experiment to obtain results on the TREC COVID queries and qrels
pt.Experiment(
    [DPH_br, BM25_br],
    dataset.get_topics('title'),
    dataset.get_qrels(),
    eval_metrics=["P.5", "P.10", "ndcg_cut.10", "map"])

# AQUAINT

In [23]:
files_AQUAINT = pt.io.find_files("/home/mrim/galuscap/data/AQUAINT")

# build the index
indexer_AQUAINT = pt.TRECCollectionIndexer("/home/mrim/galuscap/data/indexes/AQUAINT", verbose=True, blocks=False)
indexref_AQUAINT = indexer_AQUAINT.index(files_AQUAINT)

# load the index, print the statistics
index_AQUAINT = pt.IndexFactory.of(indexref_AQUAINT)
print(index_AQUAINT.getCollectionStatistics().toString())


  0%|                                                            | 0/3350 [00:00<?, ?files/s]

16:09:36.265 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/AQUAINT/disk1/AQUAINT.DTD. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
16:09:36.283 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/AQUAINT/disk1/FILE.TBL. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
16:09:36.306 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/AQUAINT/disk1/INDEX.HTML. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed



 48%|███████████████████████▌                         | 1614/3350 [08:46<04:10,  6.92files/s]

16:18:23.258 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/AQUAINT/disk2/AQUAINT.DTD. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
16:18:23.317 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/AQUAINT/disk2/FILE.TBL. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed
16:18:23.325 [main] WARN org.terrier.indexing.MultiDocumentFileCollection - TRECCollection found no documents in /home/mrim/galuscap/data/AQUAINT/disk2/INDEX.HTML. Perhaps trec.collection.class is wrongly set, TrecDocTags are incorrect, or decompression failed



100%|█████████████████████████████████████████████████| 3350/3350 [13:07<00:00,  5.06files/s]
3351files [13:07,  4.25files/s]                                                              


Number of documents: 1033461
Number of terms: 647280
Number of postings: 179114512
Number of fields: 0
Number of tokens: 282858247
Field names: []
Positions:   false



In [26]:
dataset_AQUAINT = pt.get_dataset("irds:aquaint/trec-robust-2005")
bm25 = pt.BatchRetrieve(index_AQUAINT, wmodel="BM25")
dph = pt.BatchRetrieve(index_AQUAINT, wmodel="DPH")
pt.Experiment(
    [bm25, dph],
    dataset_AQUAINT.get_topics('title'), # Might also use  'description', 'narrative'
    dataset_AQUAINT.get_qrels(),
    eval_metrics=["map"]
)

INFO:root:If you have a local copy of https://trec.nist.gov/data/robust/05/05.50.topics.txt, you can symlink it here to avoid downloading it again: /home/mrim/gonzagab/.ir_datasets/downloads/c2e722e6bdfd00f088c6f6517db564ce
[INFO] If you have a local copy of https://trec.nist.gov/data/robust/05/05.50.topics.txt, you can symlink it here to avoid downloading it again: /home/mrim/gonzagab/.ir_datasets/downloads/c2e722e6bdfd00f088c6f6517db564ce
  2%|▊                                                | 68/4016 [18:02<1:32:08,  1.40s/files]INFO:root:[starting] https://trec.nist.gov/data/robust/05/05.50.topics.txt
[INFO] [starting] https://trec.nist.gov/data/robust/05/05.50.topics.txt                      
  2%|▊                                                | 68/4016 [18:03<1:32:08,  1.40s/files]
https://trec.nist.gov/data/robust/05/05.50.topics.txt: 0.0%| 0.00/25.1k [00:00<?, ?B/s]INFO:root:[finished] https://trec.nist.gov/data/robust/05/05.50.topics.txt: [00:00] [25.1kB] [263kB/s]
         

,name,map
0,BR(BM25),0.180697
1,BR(DPH),0.199057


# Twitter 

In [17]:
files_tweets = pt.io.find_files("/home/mrim/galuscap/data/Twitter2013/Tweets2013-corpus/data")

# build the index
indexer_tweets = pt.TRECCollectionIndexer("/home/mrim/galuscap/data/indexes/tweets", verbose=True, blocks=False)
indexref_tweets = indexer_tweets.index(files_tweets)

# load the index, print the statistics
index_tweets = pt.IndexFactory.of(indexref_tweets)
print(index_tweets.getCollectionStatistics().toString())

  0%|                                                            | 0/1416 [00:00<?, ?files/s]

14:33:36.971 [main] WARN org.terrier.compression.bit.BitOutputStream - Could not open new BitOutputStream because it alleged file could not be found.
java.io.FileNotFoundException: /home/mrim/galuscap/data/indexes/tweets/data_1.direct.bf (Permission non accordÃ©e)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:298)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:237)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:126)
	at org.terrier.utility.io.LocalFileSystem.writeFileStream(LocalFileSystem.java:130)
	at org.terrier.utility.Files.writeFile(Files.java:370)
	at org.terrier.utility.Files.writeFileStream(Files.java:706)
	at org.terrier.compression.bit.BitOutputStream.<init>(BitOutputStream.java:113)
	at org.terrier.structures.bit.DirectInvertedOutputStream.<init>(DirectInvertedOutputStream.java:73)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInsta

JavaException: JVM exception occurred: Could not instantiate MetaIndexBuilder org.terrier.structures.indexing.ZstdMetaIndexBuilder java.lang.IllegalArgumentException

In [ ]:
import ir_datasets
dataset = ir_datasets.load("tweets2013-ia")
for doc in dataset.docs_iter():

In [ ]:
# TREC DL

In [20]:
files_TRECDL = pt.io.find_files("/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized")
print (files_TRECDL)
print(pt)

# build the index
indexer_TRECDL = pt.TRECCollectionIndexer("/home/mrim/galuscap/data/indexes/DL_index", verbose=True, blocks=False)
#indexref_TRECDL = indexer_TRECDL.index(files_TRECDL)

# load the index, print the statistics
#index_TRECDL = pt.IndexFactory.of(indexref_TRECDL)

['/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized/msmarco_passage_00.gz.txt', '/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized/msmarco_passage_01.gz.txt', '/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized/msmarco_passage_02.gz.txt', '/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized/msmarco_passage_03.gz.txt', '/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized/msmarco_passage_04.gz.txt', '/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized/msmarco_passage_05.gz.txt', '/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized/msmarco_passage_06.gz.txt', '/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized/msmarco_passage_07.gz.txt', '/home/mrim/data/collection/msmarco_v2/passage/msmarco_v2_passage_trec_normalized/msmarco_passage_08.gz.txt', '/home/mr

TypeError: 'NoneType' object is not callable